In [1]:
import numpy as np
from scipy.ndimage import label, generate_binary_structure
import scipy.ndimage as ndimage

In [198]:
input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [191]:
with open('day3-input.txt', 'r') as f:
    input = f.read()

In [199]:
data = np.array([ [c for c in l] for l in input.splitlines()])
data

array([['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']], dtype='<U1')

In [200]:
feats = np.where(~np.char.isnumeric(data) & (data != '.'), 1, 0)
feats

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [201]:
nums = np.where(np.char.isnumeric(data), 1, 0)
nums

array([[1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 1, 0, 0]])

In [202]:
nstructure = [[0,0,0],
 [1,1,1],
 [0,0,0]]
labeled_nums, nums_count = label(nums, structure=nstructure)
labeled_nums

array([[ 1,  1,  1,  0,  0,  2,  2,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  3,  0,  0,  4,  4,  4,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  5,  5,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  6,  6,  0],
       [ 0,  0,  7,  7,  7,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  8,  8,  8,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  9,  9,  9,  0, 10, 10, 10,  0,  0]], dtype=int32)

In [203]:
def has_adjacent(labeled_nums, i, feat):
    grouped = np.where(labeled_nums == i, 1, 0) + feat
    labeled_grouped, _ = label(grouped, structure=generate_binary_structure(2,2))
    mask = ndimage.sum(grouped, labeled_grouped , np.arange(0,1000))  > 1
    sized = mask[labeled_grouped.ravel()].reshape(labeled_grouped.shape).astype(int)
    result = sized - np.where(labeled_nums == i, 1, 0)
    return result.any(), int(''.join(data[np.where(labeled_nums == i)]))

total = 0
for i in range(1, nums_count+1):
    has, val = has_adjacent(labeled_nums, i, feats)
    if has:
        total += val
total

4361

# Part 2

In [204]:
stars = np.where(data == '*', 1, 0)
star_label, star_count = label(stars)

total = 0
for i in range(1, star_count+1):
    star = np.where(star_label == i, 1, 0)
    labeled_gear, new_count = label(labeled_nums + star,  structure=generate_binary_structure(2,2))
    if new_count != nums_count:
        gear_label_value = labeled_gear[np.where(star == 1)][0]
        # select only that label
        interest = np.where(labeled_gear == gear_label_value, 1, 0)
        # but then remove our star again
        distinct, distinct_count = label(interest - star)
        nums = []
        for i in range(1, distinct_count+1):
            num = int(''.join(data[np.where(distinct == i)]))
            nums.append(num)
        #print(nums)
        l, r = list(map(int,nums))
        total += l*r
total

467835